In [1]:
import sys

sys.path.append('..')

from utils import TRAIN, TEST, MakiKFoldBalance, foldername2_minor_class as foldername2class #foldername2class, 
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import numpy as np

from glob import glob

import seaborn as sns
import os
import json
import copy


In [2]:
NUM_K = 10
NUM_CLASSES = 11
PATH_DATASET = '../expand_double_modes'
EXP_FOLDER = 'dataset_kfold_2'

In [3]:
# First - collect all data into massive
data_path = []
labels = []
for single_folder in glob(os.path.join(PATH_DATASET, '*')):
    # Each folder - collect images
    folder_class = single_folder.split('/')[-1]
    if foldername2class.get(folder_class) is None:
        continue
    
    for single_img_path in glob(os.path.join(single_folder, '*')):
        data_path.append(single_img_path)
        labels.append(foldername2class[folder_class])

In [4]:
data_path, labels = shuffle(data_path, labels)
kf = MakiKFoldBalance(n_groups=NUM_K, shuffle_data=True)

In [5]:
# Third - save each part as config file
# i.e as json file with train and test data

for inum_kfold, (train, test) in enumerate(kf.split(data_path, labels, NUM_CLASSES)):
    # Create folder with certain dataset config
    folder_exp_single = os.path.join(EXP_FOLDER, f'{inum_kfold}_exp')
    os.makedirs(folder_exp_single, exist_ok=True)
    config_data = {
        TRAIN: (np.asarray(copy.deepcopy(data_path))[train]).tolist(),
        TEST: (np.asarray(copy.deepcopy(data_path))[test]).tolist()
    }
    
    with open(os.path.join(folder_exp_single, 'config_data.json'), 'w') as fp:
        json.dump(config_data, fp)